In [1]:
from __future__ import print_function
from __future__ import division
import json
import py_crepe
import datetime
import numpy as np
import data_helpersTL
import dataTL
import string
import pandas as pd
np.random.seed(0123)  # for reproducibility


Using Theano backend.


In [2]:
# set parameters:

subset = None

#Whether to save model parameters
save = False
model_name_path = 'params/crepe_model.json'
model_weights_path = 'params/crepe_model_weights.h5'

#Maximum length. Longer gets chopped. Shorter gets padded.
maxlen = 1014

#Model params
#Filters for conv layers
nb_filter = 128 #initially 256
#Number of units in the dense layer
dense_outputs = 512 #Initially 1024
#Conv layer kernel size
filter_kernels = [7, 7, 3, 3, 3, 3]
#Number of units in the final output layer. Number of classes.

#Compile/fit params
batch_size = 32
nb_epoch = 70

In [3]:
print('Loading data...')
#Expect x to be a list of sentences. Y to be a one-hot encoding of the
#categories.

### 515-1122-122 and 1573 with remove 6 layers
#authorlist=[121, 479 , 649 ]
#doc_id = 14706

authorlist=[ 114, 1492, 124, 1228]
doc_id = [206, 205]
cat_output = len(authorlist) #binary in the last layer

# def main(authorlist, doc_id):
    
    
((trainX, trainY), (valX, valY)) = data_helpersTL.load_ag_data(authors = authorlist, docID = doc_id)

print('Creating vocab...')
vocab, reverse_vocab, vocab_size, check = data_helpersTL.create_vocab_set()


#trainX = data_helpers.encode_data(trainX, maxlen, vocab, vocab_size, check)
#test_data = data_helpers.encode_data(valX, maxlen, vocab, vocab_size, check)

print('Build model...')

classes = len(authorlist)
import cPickle as pickle
with open('sgd.pickle', 'rb') as handle:
    sgd = pickle.load(handle)

(model) = py_crepe.build_feature_model(classes, filter_kernels, dense_outputs, maxlen,
                                       vocab_size, nb_filter, sgd, model_weights_path)
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()


Loading data...
Execution completed
Read completed
Number of rows: 44
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (13608, 2)
Author:   114  Size: 11925
Author:   124  Size:   415
Author:  1228  Size:   506
Author:  1492  Size:   762
Min: 415
Max: 11925
Authors [114, 124, 1228, 1492].
Found 1660 texts.
Found 1660 labels.
Creating vocab...
Build model...


In [4]:
vocab_size

127

In [5]:
print('Fit model...')
initial = datetime.datetime.now()
for e in xrange(nb_epoch):
    xi, yi = data_helpersTL.shuffle_matrix(trainX, trainY)
    xi_test, yi_test = data_helpersTL.shuffle_matrix(valX, valY)
    if subset:
        batches = data_helpersTL.mini_batch_generator(xi[:subset], yi[:subset],
                                                    vocab, vocab_size, check,
                                                    maxlen,
                                                    batch_size=batch_size)
    else:
        batches = data_helpersTL.mini_batch_generator(xi, yi, vocab, vocab_size,
                                                    check, maxlen,
                                                    batch_size=batch_size)

    test_batches = data_helpersTL.mini_batch_generator(xi_test, yi_test, vocab,
                                                     vocab_size, check, maxlen,
                                                     batch_size=batch_size)
    
    
    accuracy = 0.0
    loss = 0.0
    step = 1
    start = datetime.datetime.now()
    print('Epoch: {}'.format(e))
    for x_train, y_train in batches:
        feature_trainX = model.predict(x_train)
        clf.partial_fit(feature_trainX)
    
    test_accuracy = 0.0
    test_loss = 0.0
    test_step = 1
    
    
    for x_test_batch, y_test_batch in test_batches:
        feature_testX = model.predict(x_test_batch)
        test_accuracy += clf.predict(feature_testX)
        test_accuracy_avg = test_accuracy / test_step
        test_step += 1
        
    print('Epoch {}. Loss: {}. Accuracy: {}\n'.format(e, test_accuracy_avg))

Fit model...
Epoch: 0


ValueError: cannot reshape array of size 4145152 into shape (32,1008,128)
Apply node that caused the error: Reshape{3}(InplaceDimShuffle{0,2,3,1}.0, MakeVector{dtype='int64'}.0)
Toposort index: 10
Inputs types: [TensorType(float32, 4D), TensorType(int64, vector)]
Inputs shapes: [(32, 1012, 1, 128), (3,)]
Inputs strides: [(518144, 4, 518144, 4048), (8,)]
Inputs values: ['not shown', array([  32, 1008,  128])]
Outputs clients: [[InplaceDimShuffle{0,2,1,x}(Reshape{3}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/home/aman/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-6328ba9f5dac>", line 33, in <module>
    vocab_size, nb_filter, sgd, model_weights_path)
  File "py_crepe.py", line 80, in build_feature_model
    name = 'conv1', trainable = False))
  File "/home/aman/anaconda2/lib/python2.7/site-packages/ke

In [ ]:
del trainX, trainY, valX, valY